In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import speech_recognition as sr
from gtts import gTTS
import os
from pydub import AudioSegment
from pymystem3 import Mystem
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import math
import collections

def freq(it):
    r = collections.defaultdict(int)
    for i in it:
        r[i] += 1
    return dict(r)

stemmer = Mystem()

def lemm(text):
    return list(filter(lambda x: x, map(lambda x: x.strip(), stemmer.lemmatize(text.lower()))))

def keywords(text):
    normal_words = lemm(text)
    normal_stop_words = lemm(" ".join(set(stopwords.words('russian'))))
    filtered_words = list(filter(lambda x: not x in normal_stop_words, normal_words))
    return freq(filtered_words)

def normalize_keywords(kw):
    sm = math.sqrt(sum(map(lambda x: x*x, kw.values())))
    if sm == 0: return {}
    return {i: v/sm for i, v in kw.items()}

def convert(nameA, nameB):
    if os.path.exists(nameB): return
    s = AudioSegment.from_mp3(nameA)
    s.export(nameB, format='wav')

def distance(a, b):
    n, m = len(a), len(b)
    if n > m: a, b, n, m = b, a, m, n
    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]: change += 1
            current_row[j] = min(add, delete, change)
    return current_row[n]

def comp_kw(a, b, max_dist=2):
    res = {}
    for i, v in a.items():
        if max_dist > 0:
            for ii, vv in b.items():
                d = distance(i, ii)
                if d <= max_dist:
                    res[(i, ii)] = v*vv
            continue
        vv = b.get(i, None)
        if vv is None: continue
        res[(i, i)] = vv*v
    return res

def sumkw(kw):
    return sum(kw.values())

[nltk_data] Downloading package punkt to /home/aynur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aynur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
r = sr.Recognizer()
music_name = "music/Георгий Кишко - Бандито, Гангстерито.mp3"
music_name_wav = f"{music_name}.wav"
convert(music_name, music_name_wav)
au = sr.AudioFile(music_name_wav)
with au as source:
    audio = r.record(source)

In [4]:
%%time
words = r.recognize_google(audio, language="ru")
print(words)

мы бандито мы стрелянто убиванто Продам Toyota ай-банку туриста паразиты мы бандито знаменито мы 5 разъезжаться целый день Кабриолет постоянно пьем чинзано постоянно сыто-пьяно мы пиратов гастролеры сеньора и гражданка
CPU times: user 78.9 ms, sys: 35.5 ms, total: 114 ms
Wall time: 1min 11s


In [6]:
kw = keywords(words)
nkw = normalize_keywords(kw)
print(nkw)

{'бандито': 0.34299717028501764, 'стрелянто': 0.17149858514250882, 'убиванто': 0.17149858514250882, 'продавать': 0.17149858514250882, 'toyota': 0.17149858514250882, 'ай': 0.17149858514250882, '-': 0.34299717028501764, 'банка': 0.17149858514250882, 'турист': 0.17149858514250882, 'паразит': 0.17149858514250882, 'знаменитый': 0.17149858514250882, '5': 0.17149858514250882, 'разъезжаться': 0.17149858514250882, 'целый': 0.17149858514250882, 'день': 0.17149858514250882, 'кабриолет': 0.17149858514250882, 'постоянно': 0.34299717028501764, 'пить': 0.17149858514250882, 'чинзать': 0.17149858514250882, 'сыто': 0.17149858514250882, 'пьяно': 0.17149858514250882, 'пират': 0.17149858514250882, 'гастролер': 0.17149858514250882, 'сеньор': 0.17149858514250882, 'гражданка': 0.17149858514250882}
{'бандито': 2, 'стрелянто': 1, 'убиванто': 1, 'продавать': 1, 'toyota': 1, 'ай': 1, '-': 2, 'банка': 1, 'турист': 1, 'паразит': 1, 'знаменитый': 1, '5': 1, 'разъезжаться': 1, 'целый': 1, 'день': 1, 'кабриолет': 1, '

In [9]:
m = sr.Microphone(device_index=0)
with m as source:
    r.adjust_for_ambient_noise(source)

In [18]:
%%time
with m as source:
    audio = r.listen(source, phrase_time_limit=2)

CPU times: user 16.2 ms, sys: 617 µs, total: 16.8 ms
Wall time: 1.8 s


In [19]:
%%time
key = r.recognize_google(audio, language="ru")
print(key)

бандита
CPU times: user 3.24 ms, sys: 24.1 ms, total: 27.4 ms
Wall time: 6.48 s


In [20]:
req = keywords(key)
nreq = normalize_keywords(req)
nreq

{'бандит': 1.0}

In [21]:
c = comp_kw(nreq, nkw)
c

{('бандит', 'бандито'): 0.34299717028501764}

In [22]:
sumkw(c)

0.34299717028501764

In [23]:
import requests
import pydub
from pydub import AudioSegment

In [53]:
post_url = "https://stt.api.cloud.yandex.net/speech/v1/stt:recognize?lang=ru-RU&topic=general&profanityFilter=true&format=oggopus"

In [46]:
api_key = dict([j for j in [i.strip().split("=", 1) for i in open("yandex_api_key", "r").read().strip().split("\n")] if len(j) == 2])

In [47]:
headers = headers = {"Authorization": f"Api-Key {api_key['SECRET_KEY']}"}

In [82]:
data = pydub.utils.make_chunks(AudioSegment.from_mp3("music/Ирма Сохадзе - Оранжевая песенка.mp3"), chunk_length=14400)[3].export(format='ogg').read()

In [83]:
open('tmp.ogg', 'wb').write(data)

189575

In [84]:

res = requests.post(post_url, headers=headers, data=data)
if res.status_code == 200:
    return res.json()['result']
else: return ""

'оранжевый поют к нам домой очень взрослый дядя покачала головой на рисунок глядя'

In [90]:
res.status_code


200

In [48]:
from pydub import AudioSegment
import pydub

s = AudioSegment.from_mp3("music/Ирма Сохадзе - Оранжевая песенка.mp3")

In [49]:
pydub.utils.make_chunks?

In [ ]:
pydub.utils

In [4]:
text = open('word_game/_vmcl.txt', 'r').read()

In [7]:
part = " ".join(text.split(" ", 200)[:-1])

In [8]:
part

'и мир издательство литература издание осуществлено в рамках краудсорсингового проекта в один клик музей усадьба на основе электронных копий томов собрания сочинений предоставленных государственной библиотекой издание томного собрания сочинений доступно на портале и редакционные пояснения к томам собрания сочинений можно прочитать в настоящем издании нашли ошибку пожалуйста напишите нам к электронному изданию издание представляет собой электронную версию томного собрания сочинений вышедшего в свет в гг уникальное академическое издание самое полное собрание наследия давно стало библиографической редкостью году музей усадьба в сотрудничестве с государственной библиотекой и при поддержке фонда и координации совета осуществили сканирование всех томов издания для того чтобы пользоваться всеми преимуществами электронной версии чтение на современных устройствах возможность работы с текстом предстояло еще распознать более страниц этого музей музей усадьба вместе с партнером компанией открыли п

In [14]:
stemmer = Mystem()
analyzed = stemmer.analyze(part)
analyzed

[{'analysis': [{'lex': 'и', 'wt': 0.9999770357, 'gr': 'CONJ='}], 'text': 'и'},
 {'text': ' '},
 {'analysis': [{'lex': 'мир',
    'wt': 0.9991432282,
    'gr': 'S,муж,неод=(вин,ед|им,ед)'}],
  'text': 'мир'},
 {'text': ' '},
 {'analysis': [{'lex': 'издательство',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,ед|им,ед)'}],
  'text': 'издательство'},
 {'text': ' '},
 {'analysis': [{'lex': 'литература', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'литература'},
 {'text': ' '},
 {'analysis': [{'lex': 'издание',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,ед|им,ед)'}],
  'text': 'издание'},
 {'text': ' '},
 {'analysis': [{'lex': 'осуществлять',
    'wt': 1,
    'gr': 'V,пе=прош,ед,прич,кр,сред,сов,страд'}],
  'text': 'осуществлено'},
 {'text': ' '},
 {'analysis': [{'lex': 'в', 'wt': 0.9999917878, 'gr': 'PR='}], 'text': 'в'},
 {'text': ' '},
 {'analysis': [{'lex': 'рамка', 'wt': 1, 'gr': 'S,жен,неод=пр,мн'}],
  'text': 'рамках'},
 {'text': ' '},
 {'analysis': [{'lex': 'краудсорсинговый',
    'wt':

['мир',
 'издательство',
 'литература',
 'издание',
 'рамках',
 'проекта',
 'клик',
 'музей',
 'усадьба',
 'основе',
 'копий',
 'томов',
 'собрания',
 'сочинений',
 'библиотекой',
 'издание',
 'собрания',
 'сочинений',
 'портале',
 'пояснения',
 'томам',
 'собрания',
 'сочинений',
 'издании',
 'ошибку',
 'нам',
 'изданию',
 'издание',
 'собой',
 'версию',
 'собрания',
 'сочинений',
 'свет',
 'издание',
 'собрание',
 'наследия',
 'редкостью',
 'году',
 'музей',
 'усадьба',
 'сотрудничестве',
 'библиотекой',
 'поддержке',
 'фонда',
 'координации',
 'совета',
 'сканирование',
 'томов',
 'издания',
 'того',
 'преимуществами',
 'версии',
 'чтение',
 'устройствах',
 'возможность',
 'работы',
 'текстом',
 'страниц',
 'музей',
 'музей',
 'усадьба',
 'партнером',
 'компанией',
 'проект',
 'клик',
 'сайте',
 'проекту',
 'тысяч',
 'волонтеров',
 'помощью',
 'программы',
 'текст',
 'ошибки',
 'дней',
 'этап',
 'сверки',
 'месяца',
 'этапа',
 'корректуры',
 'тома',
 'произведения',
 'виде',
 'сайте

In [ ]:
import speech_recognition as sr
import time

r = sr.Recognizer()

# Words that sphinx should listen closely for. 0-1 is the sensitivity
# of the wake word.
keywords = [("привет", 1), ("олень", 1), ]

source = sr.Microphone(device_index=sr.Microphone.list_microphone_names().index('sysdefault'))


def callback(recognizer, audio):  # this is called from the background thread
    print('try recog')
    try:
        speech_as_text = recognizer.recognize_sphinx(audio, keyword_entries=keywords, language='ru')
        print(speech_as_text)

        # Look for your "Ok Google" keyword in speech_as_text
        if "привет" in speech_as_text:
            recognize_main()

    except sr.UnknownValueError:
        print("Oops! Didn't catch that")


def recognize_main():
    print("Recognizing Main...")
#   audio_data = r.listen(source)
    # interpret the user's words however you normally interpret them


def start_recognizer():
    r.listen_in_background(source, callback)
    time.sleep(1000000)


start_recognizer()

In [11]:
%pip install simpleaudio

     |████████████████████████████████| 2.0MB 496kB/s eta 0:00:01
  Created wheel for simpleaudio: filename=simpleaudio-1.0.2-cp37-cp37m-linux_x86_64.whl size=2066000 sha256=b380a5a3660f5ca3a7175749f78a093f66598886f430a90218189485873d46cf
  Stored in directory: /home/aynur/.cache/pip/wheels/6c/19/5e/8d9ca838b454227707ce995d53dfc1a4557bbdf89529ff81f2
Successfully built simpleaudio
Note: you may need to restart the kernel to use updated packages.


In [9]:
sr.Microphone.list_microphone_names().index('front')

7